<center>
    <h1>
    Keras<br>
    </h1>
</center>

In [ ]:
%%html

<!-- styles d'affichage -->
<style>
    .section_div {
        width:70%;
        height:1.5px;
        border:none;
        color:black;
        background-color:black;
        margin: auto;
        margin-top: 0px;
        margin-bottom: 0px;
    }

    .answer {
        color:blue;
    }

    .question {
        color:red;
    }

    .note {
        color:green;
        font-weight: bold;
    }
</style>

In [ ]:
#
# assure le reload de src si modifications sont faite
#
%load_ext autoreload
%autoreload 2

In [ ]:
#
# import utilitaires
#
%matplotlib inline

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns

from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
#
# import package develope pour le projet
#
import aihelpers.unsupervised as ai
import aihelpers.unsupervised.association_rules as rules

In [ ]:
#
# import keras
#
from keras.models import Model
from keras.layers import (
    BatchNormalization,
    Dense,
    Dropout,
    Input
)
from keras.optimizers import Adam
from keras.losses import (
    SparseCategoricalCrossentropy,
    BinaryCrossentropy,
    CategoricalCrossentropy
)
from keras.callbacks import Callback
from tqdm.keras import TqdmCallback

In [ ]:
#
# utilitaire keras pour entrainement et test
#
class MetricLoggerCallback(Callback):
    def __init__(self):
        # TODO: init proprietes necessaire pour logger les metriques
        pass

    def on_epoch_end(self, epoch, logs=None):
        # TODO: logger metriques durant apprentissage
        pass

def train_model(model,
          x_train, 
          y_train,
          x_validation, 
          y_validation,
          epochs=100,
          batch_size=64):
    metric_logger = MetricLoggerCallback()
    model.fit(x_train, y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(x_validation, y_validation),
              verbose=0,
              callbacks=[TqdmCallback(), metric_logger])

def test_model(model, x_test, y_test):
    # TODO
    pass

In [ ]:
#
# TODO: visualise, load, split et one hot encode data ici
#

operators = ["Avril", "Jo", "Azrael", "Lucky Luck"]
machines = ["Death", "Destruction", "Dream", "Desire", "Delirium", "Despair", "Sauron", "Deadpool"]
size = 50

data = pd.DataFrame({
    "operators": np.random.choice(operators, size=size),
    "machines": np.random.choice(machines, size=size),
})

# creation set de train, validation et test 
data_train, data_validation = train_test_split(data,
                                               train_size=0.8,
                                               random_state=42,
                                               stratify=None)
data_train, data_test = train_test_split(data_train,
                                         train_size=0.8,
                                         random_state=24,
                                         stratify=None)

data_train.reset_index(inplace=True, drop=True)
data_validation.reset_index(inplace=True, drop=True)
data_test.reset_index(inplace=True, drop=True)

In [ ]:
#
# model simple
#
inputs = Input(shape=(10, 10))

# block 0
features = BatchNormalization()(inputs)
features = Dropout(rate=0.2)(features)
features = Dense(units=512,
                 activation="relu")(features)

# block 1
features = BatchNormalization()(features)
features = Dropout(rate=0.2)(features)
features = Dense(units=256,
                 activation="relu")(features)

model = Model(inputs=inputs, outputs=features)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=CategoricalCrossentropy(from_logits=False))

model.summary()

In [ ]:
#
# entrainement du modele
#
train_model(model,
            data_train.iloc[:, :1],
            data_train.iloc[:, 1:],
            data_validation.iloc[:, :1],
            data_validation.iloc[:, 1:],
            epochs=5,
            batch_size=5)